<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [7]</a>'.</span>

**NOTE**: Here we test the performance of the STged with gene expression deconvolution methods

In [1]:

#remove(list = ls())
#define the conda env

library(reticulate)
use_condaenv("ccnet", required = TRUE)

python_env = "python_env"

In [2]:
# Source code
source("./code/STged.R")
source("./code/benchmark.R")

# Load data the generated simulated data
##  we use the FISH+ data from the mouse tissue. There are 69 spots.

In [3]:
sim_data = readRDS("./realdata/FISH+/sim_fishplus_input.RDS")

TRUE_F = sim_data$trueF
beta.type = sim_data$beta.type

cell_type = colnames(beta.type)
cell_type

[1] "astrocytes" "endo_mural" "eNeuron"    "iNeuron"    "microglia" 
[6] "Olig"

# Run the STged step by step

## Step 1: clear data

In [4]:
depthscale = 1e6
clean.only = FALSE
datax = data_process(sc_exp = sim_data$sc_exp,   sc_label = sim_data$sc_label, 
                     spot_exp = sim_data$spot_exp,  spot_loc = sim_data$spot_loc,
                     depthscale = depthscale,  gene_det_in_min_cells_per = 0.05, 
                     expression_threshold = 0,
                     nUMI =  100, verbose = FALSE, clean.only = clean.only)

## Winsorize expression values to prevent outliers  
truncate = TRUE
if(truncate){
  qt = 0.05
  datax$sc_exp  =  winsorize(x =  datax$sc_exp, qt = qt)
  datax$spot_exp  =  winsorize(x =  datax$spot_exp, qt = qt)
}


##  True cell type-specific gene expression data

In [5]:
gene_sec = rownames(datax$sc_exp)
for(i in 1:length(cell_type)){
  
  TRUE_F[[i]] = TRUE_F[[i]][gene_sec,colnames(datax$spot_exp )]
  
}
# normaliztion of the true cell type-specific gene expression data
trueF = res_library_size_normaization(estres = TRUE_F, depthscale = depthscale )

## Step 2: construct spatial correlation structures

In [6]:
cat("Construct spatial correlation", "\n")
L.mat = dis_weight(spot_loc = datax$spot_loc, spot_exp = datax$spot_exp, k = 6, 
                   quantile_prob_bandwidth = 1/3, method = "Hex", 
                   coord_type = "grid")

Construct spatial correlation 


## Step 3: construct reference gene matrix

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [7]:

cat("Construct reference gene matrix", "\n")

ref_exp = create_group_exp(sc_exp = datax$sc_exp, sc_label = datax$sc_label)
ref_exp = sweep(temp, 2, colSums(ref_exp), '/')

ref_exp = ref_exp[rownames(datax$spot_exp), cell_type]
colnames(ref_exp )

beta.type = beta.type[colnames(datax$spot_exp),]



Construct reference gene matrix 


ERROR: Error in eval(expr, envir, enclos): object 'temp' not found


# Sensitive analysis

In [ ]:

lambda_values1 <- c(0.001,0.01,  0.1,1,5)
lambda_values2 <- c( 0.001,0.01,0.123 ,1,5)


# Generate all combinations of lambda1 and lambda2 using expand.grid
parameter_grid <- expand.grid(lambda1 = lambda_values1, lambda2 = lambda_values2)

sens_STged = matrix(list(), length(methods) ,1)
for(ii in 1:nrow(parameter_grid)){
  
  cat("Run the STged", "\n")
  
  start_time <- Sys.time()
  stged.est =MUR.STged(srt_exp = datax$spot_exp, ref_exp = ref_exp, 
                        beta.type = beta.type,   W = L.mat$dis_weight, 
                        lambda1 = parameter_grid[ii,1], lambda2 = parameter_grid[ii,2],
                        cutoff = 0.05, 
                        epsilon = 1e-5)

  end_time <- Sys.time()
  sens_STged[[ii]] = stged.est$V.hat
  cat("Run time of STged", end_time - start_time,"\n")

}

saveRDS(sens_STged, file = "./results/FISH+/True_spot_decon_sens_VAR.RDS")

saveRDS(parameter_grid, file = paste0("./results/FISH+/True_cell_exp_tuning_values_VAR.RDS"))